This is the builder pattern
y luego el nombre

In [1]:
from pyspark.sql import SparkSession

#config("spark.sql.repl.eagerEval.enabled","True").

spark = (
    SparkSession.
    builder.
    appName("Analysing the vocabulary of Pride and Prejudice.").
    getOrCreate()
)

Ingest transform and load

Basicamente tienes un programa corriendo ya

ya no lo esta creando el programa

ese programa se llama sparksession

aqui tu lo haces

In [2]:
spark

In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=Analysing the vocabulary of Pride and Prejudice.>

spark session es un wrapper sobre sparkcontext

In [4]:
spark.sparkContext.setLogLevel("WARN")

### What are the most popular words used in the English language?

- Read the input data plain text
- Tokenize

RDD records

Data Frame

optimized row columnar

In [11]:

book=spark.read.text("Gutenberg/1342-0.txt")

book

DataFrame[value: string]

In [6]:
book.dtypes

[('value', 'string')]

In [7]:
spark.__doc__

'The entry point to programming Spark with the Dataset and DataFrame API.\n\n    A SparkSession can be used to create :class:`DataFrame`, register :class:`DataFrame` as\n    tables, execute SQL over tables, cache tables, and read parquet files.\n    To create a :class:`SparkSession`, use the following builder pattern:\n\n    .. versionchanged:: 3.4.0\n        Supports Spark Connect.\n\n    .. autoattribute:: builder\n       :annotation:\n\n    Examples\n    --------\n    Create a Spark session.\n\n    >>> spark = (\n    ...     SparkSession.builder\n    ...         .master("local")\n    ...         .appName("Word Count")\n    ...         .config("spark.some.config.option", "some-value")\n    ...         .getOrCreate()\n    ... )\n\n    Create a Spark session with Spark Connect.\n\n    >>> spark = (\n    ...     SparkSession.builder\n    ...         .remote("sc://localhost")\n    ...         .appName("Word Count")\n    ...         .config("spark.some.config.option", "some-value")\n    .

In [8]:
book.show()

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|                    |
|Title: Pride and ...|
|                    |
| Author: Jane Austen|
|                    |
|Posting Date: Aug...|
|Release Date: Jun...|
|Last Updated: Mar...|
|                    |
|   Language: English|
|                    |
|Character set enc...|
|                    |
+--------------------+
only showing top 20 rows



In [9]:
book.show(n=20,truncate=False,vertical=True)

-RECORD 0---------------------------------------------------------------------
 value | The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen   
-RECORD 1---------------------------------------------------------------------
 value |                                                                      
-RECORD 2---------------------------------------------------------------------
 value | This eBook is for the use of anyone anywhere at no cost and with     
-RECORD 3---------------------------------------------------------------------
 value | almost no restrictions whatsoever.  You may copy it, give it away or 
-RECORD 4---------------------------------------------------------------------
 value | re-use it under the terms of the Project Gutenberg License included  
-RECORD 5---------------------------------------------------------------------
 value | with this eBook or online at www.gutenberg.org                       
-RECORD 6-------------------------------------------

In [81]:
from pyspark.sql.functions import col,split, explode, lower

#book.select(book.value).show()
#book.select(book["value"]).show()



#el alias se pone dentro de la columna
#split es un col te regresa un col con nombre cambiado!
#select te da un dataframe de un col

lines=(
    book.select(
        split(
            col("value"), 
            " "
        ).alias("line")
    ).alias("Ale")
    #aqui no vas a saber que columna quieres!
)

words=(
    lines.select(
        explode(
            col("line")
        ).
        alias("word")
    )
)

words.show(truncate=False)


+----------+
|word      |
+----------+
|The       |
|Project   |
|Gutenberg |
|EBook     |
|of        |
|Pride     |
|and       |
|Prejudice,|
|by        |
|Jane      |
|Austen    |
|          |
|This      |
|eBook     |
|is        |
|for       |
|the       |
|use       |
|of        |
|anyone    |
+----------+
only showing top 20 rows



In [87]:
words_lower=(
    words.select(
        lower(
            col("word")
        ).alias("word_lower")
    )
)

words_lower.show()

+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
+----------+
only showing top 20 rows



In [106]:
from pyspark.sql.functions import regexp_extract

#aqui se quitan las comas, 
words_clean=(
    words_lower.select(
        regexp_extract(
            str=col("word_lower"),
            pattern="[a-z]+",
            #se extrae la primer ocurrencia!
            idx=0
        ).alias("word")
    )
)

words_clean.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|         |
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
+---------+
only showing top 20 rows



In [114]:
words_nonull=words_clean.filter(col("word")!="")
words_nonull.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
+---------+
only showing top 20 rows



In [122]:
results=(
    words_nonull.groupBy(
        col("word")
    ).count()
)

results.show()

+-------------+-----+
|         word|count|
+-------------+-----+
|       online|    4|
|         some|  209|
|        still|   72|
|          few|   72|
|         hope|  122|
|        those|   60|
|     cautious|    4|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
+-------------+-----+
only showing top 20 rows



In [129]:
results.orderBy(col("count"), ascending=False).show(10)

+----+-----+
|word|count|
+----+-----+
| the| 4496|
|  to| 4235|
|  of| 3719|
| and| 3602|
| her| 2223|
|   i| 2052|
|   a| 1997|
|  in| 1920|
| was| 1844|
| she| 1703|
+----+-----+
only showing top 10 rows



In [131]:
results.show()

+-------------+-----+
|         word|count|
+-------------+-----+
|       online|    4|
|         some|  209|
|        still|   72|
|          few|   72|
|         hope|  122|
|        those|   60|
|     cautious|    4|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
+-------------+-----+
only showing top 20 rows



In [134]:
results.write.csv("Data/simple_count.csv", mode="overwrite")